In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from os import walk
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pickle
import h5py

Loading the dataset and normalizing

In [ ]:
loc = 'bbg10' # location
modelID = 5
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 
include_wave = 'no'
duration = '24M'

durations = ['3M','6M','9M','12M','15M','18M','21M','24M']
train_end_dates = ['2018-03-31 23:50:00+00:00', '2018-06-30 23:50:00+00:00', '2018-09-30 23:50:00+00:00', 
                  '2018-12-31 23:50:00+00:00', '2019-03-31 23:50:00+00:00', '2019-06-30 23:50:00+00:00',
                  '2019-09-30 23:50:00+00:00', '2019-12-31 23:50:00+00:00']
train_end_date = train_end_dates[durations.index(duration)]

# Laod train data 
train_input = pd.read_pickle('DATA/train_input')
train_output = pd.read_pickle('DATA/train_output')
index = train_input.columns

train_input = train_input.loc['2018-01-01 00:00:00+00:00':train_end_date]
train_output= train_output.loc['2018-01-01 00:00:00+00:00':train_end_date]
# print(train_input.shape)

# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)
inputn = pd.DataFrame(std_scaler.transform(train_input), columns=train_input.columns) 
# inputn is still a daraframe with numeric index
outputn = train_output/10**5  #  change of units, outputn is still a daraframe with time index

Retrive features based on the modelID

In [ ]:
index1 = pd.core.indexes.base.Index([]) # create a blank index array
if include_wave == 'yes': 
    index1 = index1.append(index[0:3])
if modelID == 1: # Acc17
    index1 = index1.append(index[[3,4,9,10,15,16]])
if modelID == 2: # Acc38
    index1 = index1.append(index[[5,6,11,12,17,18]])
if modelID == 3: # Acc77
    index1 = index1.append(index[[7,8,13,14,19,20]])
if modelID == 4: # Acc17&38
    index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
if modelID == 5: # Acc17&38&77
    index1 = index1.append(index[3:21])

index1 = index1.append(index[21:]) # SCADA
X = inputn[index1].values
Y = outputn.values

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
def NLL(y, distr): 
  return -distr.log_prob(y) 

def normal_sp(params): 
  return tfp.distributions.Normal(loc=params[:,0:2], scale=1e-3 
                                  + tf.math.softplus(0.05 * params[:,2:4]))# both parameters are learnable

kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)

inputs = tf.keras.layers.Input(shape=(X.shape[1],))

hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(inputs)
hidden = tfp.layers.DenseFlipout(64,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
params = tfp.layers.DenseFlipout(4,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn)(hidden)
dist = tfp.layers.DistributionLambda(normal_sp)(params)


model = Model(inputs=inputs, outputs=dist)
model.compile(Adam(learning_rate=0.0003), loss=NLL) 

model_params = Model(inputs=inputs, outputs=params)
model.summary()

In [ ]:
# Records the weights throughout the training process
epoch_history = []
# A custom callback
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback
class MyCallback1(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        weights = model.get_weights()
        file = h5py.File('Weights/03_UncertaintyEvolution/%d.h5' % (epoch), 'w')
        for i in range(len(weights)):
           file.create_dataset('weight' + str(i), data=weights[i])
        file.close()
        epoch_history.append(weights)
callback1 = MyCallback1()

batch_history = []
class MyCallback2(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs):
        weights = model.get_weights()
        batch_history.append(weights)
callback2 = MyCallback2()

callback3 = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True)


In [ ]:
#train the model
epoch = 2000
batch_size = 1024
history = model.fit(X, Y, batch_size=batch_size, epochs=epoch, verbose=1, validation_split = 0, 
          callbacks = [callback3]); # To save weight history, add callback1 and/or callback2

# save the final weights
file = h5py.File('Weights/01_SensorPlacementTest1/BNNModel%d_IncWave%s.h5' % (modelID, include_wave), 'w')
weight = model.get_weights()
for i in range(len(weight)):
   file.create_dataset('weight' + str(i), data=weight[i])
file.close()

In [ ]:
# Uncomment to save weight history
# from scipy import io
# io.savemat('Weights/02_PredictionModel1/Model%d_IncWave%s_%s_EpochHistory.mat'% (modelID, include_wave, duration), {"weights_history": epoch_history })
# io.savemat('Weights/02_PredictionModel1/Model%d_IncWave%s_%s_BatchHistory.mat'% (modelID, include_wave, duration), {"weights_history": batch_history })

In [ ]:
def compute_predictions_pbnn(model, samples):
    prediction_distribution= model(samples)
    prediction_mean = np.squeeze(prediction_distribution.mean().numpy())
    prediction_stdv = np.squeeze(prediction_distribution.stddev().numpy())

    # The 95% CI is computed as mean ± (1.96 * stdv)
    upper = (prediction_mean + (1.96 * prediction_stdv))
    lower = (prediction_mean - (1.96 * prediction_stdv))
  
    return prediction_mean, prediction_stdv, upper, lower

def loglikelihood(y, loc, scale):
    dist = tfp.distributions.Normal(loc, scale)
    return dist.log_prob(y)

In [ ]:
Nll_Mtl = np.zeros([len(X)])
Nll_Mtn = np.zeros([len(X)])
for j in range(100):
    prediction_mean, prediction_stdv, upper, lower = compute_predictions_pbnn(model, X)
    nll = loglikelihood(Y, prediction_mean, prediction_stdv)
    # running average over simulations
    Nll_Mtl += nll[:,0]/100
    Nll_Mtn += nll[:,1]/100

# Average over trainset
ECov_Mtl = np.mean(Nll_Mtl) 
ECov_Mtn = np.mean(Nll_Mtn)
print(ECov_Mtl)
print(ECov_Mtn) 